In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

DATA_FILEPATH = "../data/raw/Bloomberg ETFs Cleaned.xlsx"

In [2]:
rawData_df = pd.read_excel(DATA_FILEPATH, sheet_name='all_etf', index_col=0, parse_dates=True)
rawData_df.columns = [x.split()[0] for x in rawData_df.columns]
rawData_df

,SPY,VTI,VXUS,ACWI,EEM,AGG,BND,GOVT,LQD,BNDX
Date,,,,,,,,,,
1994-01-03,46.4688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994-01-04,46.6563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994-01-05,46.7500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994-01-06,46.7500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994-01-07,47.0313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2024-10-11,579.5800,286.11,64.110,120.16,46.36,99.50,73.82,23.020,110.81,49.94
2024-10-14,584.3200,288.34,64.145,120.78,46.17,99.41,73.77,23.000,110.82,49.94
2024-10-15,579.7800,286.36,63.130,119.52,45.19,99.81,74.06,23.095,111.36,50.09


In [4]:
rets = rawData_df.pct_change()
rets

,SPY,VTI,VXUS,ACWI,EEM,AGG,BND,GOVT,LQD,BNDX
Date,,,,,,,,,,
1994-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994-01-04,0.004035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994-01-05,0.002008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994-01-06,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994-01-07,0.006017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2024-10-11,0.005988,0.007749,0.005805,0.006281,0.007169,0.000302,0.000135,0.000435,-0.000180,0.000401
2024-10-14,0.008178,0.007794,0.000546,0.005160,-0.004098,-0.000905,-0.000677,-0.000869,0.000090,0.000000
2024-10-15,-0.007770,-0.006867,-0.015824,-0.010432,-0.021226,0.004024,0.003931,0.004130,0.004873,0.003004


In [ ]:
# Calc MA, for long and short windows
# think of when to open/close posn
# backtesting framework
# run on SPY / 60/40 portfolio
# evaluate with performance/risk metrics